In [ ]:
# -*- coding: utf-8 -*-
__author__ = 'maxbrainrus'

import struct
from datetime import timedelta
import gzip
from datetime import date
import numpy as np

zerodate = date(1970,1,1)

# Example:
# drawData.blocksData, drawData.gxc, drawData.gyc, drawData.gzc, drawData.volumeType, drawData.gridX, drawData.gridY, drawData.gridZ = analizeDomFile(
#     domFile)
# drawData.dataIsBroken = True
# drawData.temperData = getTempDataFromFileWithName(fileName, drawData)

def getTempDataFromFileObject(f, drawData, gz=False):
    blocksData, gxc, gyc, gzc = drawData.blocksData, drawData.gxc, drawData.gyc, drawData.gzc

    ver = struct.unpack(4 * 'B', f.read(4))  # first 4 bytes
    if ver[1] == 2:
        floatType = np.float32
    elif ver[1] == 3:
        floatType = np.float64

    saveTime = struct.unpack('d', f.read(8))  # time
    saveTimeAsString = str(zerodate + timedelta(saveTime[0]))
    resultTemperature = np.zeros((gzc - 1, gyc - 1, gxc - 1), dtype=floatType)
    resultLiquidWaterPercentage = np.zeros((gzc - 1, gyc - 1, gxc - 1), dtype=floatType)
    blocksCount = len(blocksData)
    for i in range(blocksCount):
        data = blocksData[i]
        xc = data['xc']
        yc = data['yc']
        zc = data['zc']
        total = xc * yc * zc
        if gz:
            buff = f.read(total * np.dtype(floatType).itemsize)
            temperature = np.frombuffer(buff, dtype=floatType, count=total)

        else:
            temperature = np.fromfile(f, dtype=floatType, count=total)

        temperature = temperature.reshape((zc, yc, xc))
        offsetX = data['offsetX']
        offsetY = data['offsetY']
        offsetZ = data['offsetZ']
        # struct.unpack('f'*total, f.read(4*total))  # Enthalpy
        f.read(4 * total)
        # struct.unpack('B'*total, f.read(total))  # AtPhaseTransition
        f.read(total)

        if gz:
            buff = f.read(total * np.dtype(floatType).itemsize)
            liquidWaterPercentage = np.frombuffer(buff, dtype=floatType, count=total)

        else:
            liquidWaterPercentage = np.fromfile(f, dtype=floatType, count=total)

        # liquidWaterPercentage = np.fromfile(f, dtype=floatType, count=total)
        liquidWaterPercentage = liquidWaterPercentage.reshape((zc, yc, xc))
        resultTemperature[offsetZ:offsetZ + zc, offsetY:offsetY + yc, offsetX:offsetX + xc] = temperature[:, :, :]
        resultLiquidWaterPercentage[offsetZ:offsetZ + zc, offsetY:offsetY + yc,
        offsetX:offsetX + xc] = liquidWaterPercentage[:, :, :]
        # Sets a NaN values for caverns
        boolArr = drawData.volumeType % 2 == 1
        resultTemperature[boolArr] = np.NaN
        resultLiquidWaterPercentage[boolArr] = np.NaN
        # shape = drawData.volumeType.shape
        # for x in range(shape[0] - 1):
        #    for y in range(shape[1] - 1):
        #        for z in range(shape[2] - 1):
        #            if drawData.volumeType[x, y, z] % 2 == 1:
        #                resultTemperature[x, y, z] = np.NaN
        #                resultLiquidWaterPercentage[x, y, z] = np.NaN

    return resultTemperature, resultLiquidWaterPercentage, saveTimeAsString

#without Qt
def getTempDataFromFileWithName(fileName, drawData):
        with open(str(fileName), 'rb') as f:
            resultTemperature, resultLiquidWaterPercentage, saveTimeAsString = getTempDataFromFileObject(f, drawData)

        print("Data loaded")
        return {
            'temperature3d': resultTemperature,
            'liquidWaterPercentage3d': resultLiquidWaterPercentage,
            'savetime': saveTimeAsString
        }

def getTempDataFromFile(info, drawData):
    if info.suffix() == 'bin':
        with open(str(info.absoluteFilePath()), 'rb') as f:
            resultTemperature, resultLiquidWaterPercentage, saveTimeAsString = getTempDataFromFileObject(f, drawData)
    elif info.suffix() == 'gz':
        with gzip.open(str(info.absoluteFilePath()), 'rb') as zf:

            resultTemperature, resultLiquidWaterPercentage, saveTimeAsString = getTempDataFromFileObject(zf, drawData,
                                                                                                         gz=True)

    print("Data loaded")
    return {
        'temperature3d': resultTemperature,
        'liquidWaterPercentage3d': resultLiquidWaterPercentage,
        'savetime': saveTimeAsString
    }

def analizeDomFile(domFile):
    blocksData = []
    gxc, gyc, gzc = None, None, None
    volumeType = None
    gridX, gridY, gridZ = None, None, None

    with open(domFile, 'rb') as f:
        # struct.unpack(3*'B', f.read(3))
        f.read(3)
        # struct.unpack(7*'f', f.read(7*4))
        f.read(7 * 4)
        # skip 7 float32 FinishTime, TimeStep, saves per month, IceSpecHeat,
        # WaterSpecHeat, WaterLatentHeat, IceVolumetricExpansion
        gxc, gyc, gzc = struct.unpack(3 * 'I', f.read(3 * 4))

        volumeType = np.zeros((gzc - 1, gyc - 1, gxc - 1), dtype=np.uint16)
        gridX = struct.unpack('f' * gxc, f.read(4 * gxc))
        gridY = struct.unpack('f' * gyc, f.read(4 * gyc))
        gridZ = struct.unpack('f' * gzc, f.read(4 * gzc))
        # print gridX, gridY, gridZ
        blocksCount = struct.unpack('I', f.read(4))
        blocksCount = blocksCount[0]

        for i in range(blocksCount):
            # struct.unpack(2*'I', f.read(2*4))
            f.read(2 * 4)
            offsetX, offsetY, offsetZ = struct.unpack(3 * 'I', f.read(3 * 4))
            xc, yc, zc = struct.unpack(3 * 'I', f.read(3 * 4))
            total = xc * yc * zc
            blocksData.append({
                'offsetX': offsetX,
                'offsetY': offsetY,
                'offsetZ': offsetZ,
                'xc': xc,
                'yc': yc,
                'zc': zc
            })
            # struct.unpack('f'*(xc+yc+zc), f.read(4*(xc+yc+zc)))  # block grid
            f.read(4 * (xc + yc + zc))
            blockVolumeType = np.fromfile(f, dtype=np.uint16, count=total)  # volume type
            blockVolumeType = blockVolumeType.reshape((zc, yc, xc))

            volumeType[offsetZ:offsetZ + zc, offsetY:offsetY + yc, offsetX:offsetX + xc] = blockVolumeType[:, :, :]

            # struct.unpack(total*'H', f.read(total*2))  # heat source number
            f.read(total * 2)
            p = 2 * xc * yc + 2 * xc * zc + 2 * yc * zc
            # struct.unpack(p*'H', f.read(p*2))  # border type
            f.read(p * 2)

    return blocksData, gxc, gyc, gzc, volumeType, gridX, gridY, gridZ

